# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
sys.path.append("../../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api, ML_pipeline

db_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

cpu is available.


In [2]:
# feature list와 data_y_flag는 model_info에서 받아와야 합니다.
# regression
param1 = {
    'model_name':'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_'
}

# forecasting
param2 = {
    'model_name':'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_LSTM_rg_'
}

# classification
param3 = {
    'model_name':'classification_actionPattern_cleanLevel0_trainX_classification_LSTM_cf_'
}

### (Optional) Get np.array data for inference

In [3]:
import numpy as np
np_data = np.load('infer_forecasting_array.npy') # infer_regression_array , infer_forecasting_array , infer_classification_array
infer_np = np_data[0]
infer_np.shape

(24, 3)

In [4]:
infer_np

array([[0.72945455, 0.50641026, 0.29019683],
       [0.728     , 0.48878205, 0.27009841],
       [0.73090909, 0.47115385, 0.25      ],
       [0.73163636, 0.48557692, 0.2327411 ],
       [0.73381818, 0.5       , 0.21548221],
       [0.73527273, 0.51442308, 0.19822331],
       [0.73381818, 0.52884615, 0.18096441],
       [0.72945455, 0.54326923, 0.16370551],
       [0.72      , 0.55769231, 0.14644662],
       [0.71709091, 0.54861111, 0.1332034 ],
       [0.71709091, 0.53952991, 0.11996018],
       [0.71709091, 0.53044872, 0.10671696],
       [0.72      , 0.52136752, 0.09347374],
       [0.71709091, 0.51228632, 0.08023052],
       [0.71709091, 0.50320513, 0.06698731],
       [0.71709091, 0.48450855, 0.05866227],
       [0.71709091, 0.46581197, 0.05033724],
       [0.71345455, 0.44711538, 0.0420122 ],
       [0.70981818, 0.4284188 , 0.03368716],
       [0.70618182, 0.40972222, 0.02536213],
       [0.70181818, 0.39102564, 0.01703709],
       [0.704     , 0.38354701, 0.01419758],
       [0.

### get model parameter for inference

In [5]:
params = param2

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [6]:
# get model meta
model_name = params['model_name']
meta_file_name = "./meta.json"
model_meta_db_style ='document'
from Clust.clust.ML.tool import meta
#model_meta = meta.read_model_meta_from_mongodb(mongodb_client, 'model', 'meta', model_name)
model_meta = meta.read_model_meta_from_local(meta_file_name, model_name)
# model_meta = meta.read_model_meta_by_db_style(model_meta_db_style, meta_file_name, model_name)

In [7]:
model_meta['model_info']['train_parameter']['batch_size'] = 1

In [8]:
model_meta

{'ingestion_param_X': {'bucket_name': 'integration',
  'ms_name': 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX',
  'feature_list': ['Temperature', 'out_temp', 'sin_hour']},
 'ingestion_param_y': {'bucket_name': 'integration',
  'ms_name': 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainy',
  'feature_list': ['Temperature']},
 'data_y_flag': False,
 'scaler_param': {'scaler_flag': 'scale',
  'scale_method': 'minmax',
  'scaler_path': './scaler/',
  'scaler_file_path': {'XScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX/minmax/c1b524a5c854f6e561c4764b1dfb4e6f/scaler.pkl'},
   'yScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainy/minmax/2ef658edf359871df2a5d6f044a18200/scaler.pkl'}}},
 'transform_param': {'data_clean_option': False,
  'split_mode': 'step_split',
  'past_step': 24,
  'future_step': 2,
  'nan_process_in

In [9]:
infer_X, scaler_y = ML_api.infer_data_preparation(model_meta, infer_np)

/home/leezy/anaconda3/envs/clust/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/leezy/anaconda3/envs/clust/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/leezy/anaconda3/envs/clust/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
infer_X.shape

(24, 3)

In [11]:
infer_X

array([[-0.02001983,  0.23738494,  0.64509841],
       [-0.02007273,  0.23681994,  0.63504921],
       [-0.01996694,  0.23625493,  0.625     ],
       [-0.0199405 ,  0.23671721,  0.61637055],
       [-0.01986116,  0.23717949,  0.6077411 ],
       [-0.01980826,  0.23764177,  0.59911165],
       [-0.01986116,  0.23810404,  0.59048221],
       [-0.02001983,  0.23856632,  0.58185276],
       [-0.02036364,  0.2390286 ,  0.57322331],
       [-0.02046942,  0.23873754,  0.5666017 ],
       [-0.02046942,  0.23844647,  0.55998009],
       [-0.02046942,  0.23815541,  0.55335848],
       [-0.02036364,  0.23786434,  0.54673687],
       [-0.02046942,  0.23757328,  0.54011526],
       [-0.02046942,  0.23728222,  0.53349365],
       [-0.02046942,  0.23668297,  0.52933114],
       [-0.02046942,  0.23608372,  0.52516862],
       [-0.02060165,  0.23548447,  0.5210061 ],
       [-0.02073388,  0.23488522,  0.51684358],
       [-0.02086612,  0.23428597,  0.51268106],
       [-0.02102479,  0.23368672,  0.508

In [12]:
prediction_result = ML_api.ML_inference(model_meta, infer_X, scaler_y)

inference data shape: torch.Size([24, 3])

Start inference

** Dimension of result for inference dataset = (24,)


In [13]:
prediction_result

,Temperature
0,2.498806
1,2.495231
2,2.491855
3,2.488950
4,2.486111
5,2.483241
6,2.480243
7,2.477117
8,2.473765
9,2.471340
